In [ ]:
! git clone https://github.com/cyizhuo/CUB-200-2011-dataset.git

In [ ]:
## FineTuning VGG16 on CUB-Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dir = '/content/CUB-200-2011-dataset/train'
val_dir = '/content/CUB-200-2011-dataset/test'

# Define transformations for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Load the CUB dataset
image_datasets = {x: datasets.ImageFolder(root=train_dir if x == 'train' else val_dir, transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}

# Load pre-trained VGG16
vgg16 = models.vgg16(pretrained=True)

# Freeze all layers
for param in vgg16.parameters():
    param.requires_grad = False

# Unfreeze some convolutional layers
for i, param in enumerate(vgg16.features.parameters(), 1):
    if i > 15:  # Adjust the number of layers to unfreeze as needed
        param.requires_grad = True

# Modify the classifier
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features, 200)  # Set to 200 for CUB dataset


# Move model to device
vgg16 = vgg16.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    train_accuracy = correct_train / total_train

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}")

    # Evaluation loop
    vgg16.eval()
    correct_val = 0
    total_val = 0
    val_running_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloaders['val']:
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item() * images.size(0)

    val_epoch_loss = val_running_loss / len(dataloaders['val'].dataset)
    val_accuracy = correct_val / total_val

    print(f"Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

# Print final test accuracy
print(f"Final Test Accuracy: {val_accuracy:.2%}")
